In [ ]:
!pip install --upgrade accelerate
!pip install sentencepiece
!pip install transformers==4.29.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 6.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.7 MB/s eta 0:00:00


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [ ]:
import itertools
from tqdm import tqdm
# Define the updated product categories, sizes, and product names
product_categories = ["Televisions", "Telephones", "Laptop", "Tablets"]
sizes = ["32", "55", "65", "42", "24"]
product_names = ["Samsung", "LG", "Sony", "Azuz", "Apple", "HP", "Dell", "MSI"]
# Define region-to-city mapping
region_cities_mapping = {
    "Tunisia": ["Carthage", "La Médina", "Bab Bhar"],
    "Bizerte": ["Bizerte Nord", "Jarzouna", "Bizerte Sud"],
    "Nabeul": ["Nabeul", "Dar Chaâbane Elfehri", "Béni Khiar"],
    "Jendouba": ["Jendouba", "Jendouba Nord", "Boussalem"],
    "Ariana": ["Ariana Ville", "Soukra", "Raouède"],
    "Kef": ["Kef Ouest", "Kef Est", "Nebeur"],
    "Siliana": ["Siliana Nord", "Siliana Sud", "Bouarada", "Gâafour" ],
    "Zaghouan": ["Zaghouan", "Zériba", "Bir Mecharga", "Fahs", "Nadhour", "Saouaf"],
    "Sousse": ["Sousse Ville", "Zaouia Ksiba Thrayat", "Sousse Ryadh",],
    "Kairouan": ["Kairouan Nord", "Kairouan Sud", "Chebika"],
    "Monastir": ["Monastir", "Ouerdanine", "Sahline", "Zéramdine"],
    "Kasserine": ["Kasserine Nord", "Kasserine Sud", "Azzouhour", ],
    "Mahdia": ["Mahdia", "Boumerdes", "Ouled Chamekh"],
    "Sidi Bouzid": ["Sidi Bouzid Ouest", "Sidi Bouzid Est"],
    "Sfax": ["Sfax Ville", "Sfax Ouest", "Sakiet Ezzit",],
    "Gafsa": ["Gafsa Nord", "Sidi Aich", "El Ksar",],
    "Gabès": ["Gabès Ville", "Gabès Ouest", "Gabès Sud"],
    "Kébili": ["Kébili Sud", "Kébili Nord", "Souk El Ahad", ],
    "Tozeur": ["Tozeur", "Dégueche", "Tameghza", "Nefta", "Hezoua"],
    "Medenine": ["Mednine Nord", "Mednine Sud", "Béni Khedach"],
    "Tataouine": ["Tataouine Nord", "Tataouine Sud", "Smar"]
}

default_value = None

# Define possible values for each placeholder
regions = list(region_cities_mapping.keys())
cities = list(itertools.chain.from_iterable(region_cities_mapping.values()))
# Generate all possible value combinations
value_combinations = list(itertools.product(regions + [default_value], cities + [default_value],
                                           product_categories + [default_value], sizes + [default_value],
                                           product_names + [default_value]))


# Define prompt templates
input_prompt_template = "Find [[PC]] options [[REGION]] in [[CITY]] with a size of [[Size]] and brand [[PN]]."
output_prompt_template = "What [[PC]] options are available [[REGION]] in [[CITY]] with a size of [[Size]] and brand [[PN]]?"

default_value = None
# Generate queries with placeholders replaced
input_output_pairs = []

for combination in tqdm(value_combinations):
    region, city, pc, size, pn = combination

    # Input prompt engineering
    input_prompt = input_prompt_template.replace("[[PC]]", str(pc)).replace("[[CITY]]", str(city)).replace("[[Size]]", str(size)).replace("[[PN]]", str(pn)).replace("[[REGION]]", str(region))

    # Output prompt engineering
    output_prompt = output_prompt_template
    if pc is None:
        output_prompt = output_prompt.replace(" [[PC]]", "")
    else:
        output_prompt = output_prompt.replace("[[PC]]", pc)

    if city is None:
        output_prompt = output_prompt.replace(" in [[CITY]]", "")
    else:
        output_prompt = output_prompt.replace("[[CITY]]", city)

    if size is None:
        output_prompt = output_prompt.replace(" with a size of [[Size]]", "")
    else:
        output_prompt = output_prompt.replace("[[Size]]", size)

    if pn is None:
        output_prompt = output_prompt.replace(" and brand [[PN]]", "")
    else:
        output_prompt = output_prompt.replace("[[PN]]", pn)

    if region is None:
        output_prompt = output_prompt.replace("[[REGION]]", "")
    else:
        output_prompt = output_prompt.replace("[[REGION]]", region)

    input_output_pairs.append({"input": input_prompt, "output": output_prompt})




100%|██████████| 415800/415800 [00:01<00:00, 317115.56it/s]


In [ ]:
import pandas as pd
df = pd.DataFrame(input_output_pairs)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:

import torch
import re
from tqdm import tqdm
# Define the fine-tuning data
train_data = []
for index, row in df.iterrows():
    input_text = row["input"]
    target_text = row["output"]
    train_data.append({'input_text':
                       input_text, 'target_text': target_text})

# Initialize the tokenizer and model

def preprocess_text(text):
    if not isinstance(text, (str, bytes)):
        text = str(text)
    # Remove leading and trailing whitespaces
    text = text.strip()
    # Remove multiple consecutive whitespaces
    text = re.sub(r"\s+", " ", text)
    # Remove special characters except '?', ',', '.', and "'"
    text = re.sub(r"[^a-zA-Z0-9\s?,.'\"]", "", text)
    return text


In [ ]:
preprocessed_pairs = [(preprocess_text(example['input_text']), preprocess_text(example['target_text'])) for example in train_data]

In [ ]:
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(preprocessed_pairs, test_size=0.2, random_state=42)

In [ ]:
train_data

In [ ]:
# Prepare the training data
def preprocess_data(examples):
    input_texts = [example[0] for example in examples]  # Extract the input text from the tuple
    target_texts = [example[1] for example in examples]  # Extract the target text from the tuple
    print(input_texts[0])
    input_encodings = tokenizer.batch_encode_plus(input_texts, padding=True, truncation=True, return_tensors='pt')
    target_encodings = tokenizer.batch_encode_plus(target_texts, padding=True, truncation=True, return_tensors='pt')

    input_ids = input_encodings['input_ids']
    attention_mask = input_encodings['attention_mask']
    target_ids = target_encodings['input_ids']
    target_attention_mask = target_encodings['attention_mask']

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'target_ids': target_ids,
        'target_attention_mask': target_attention_mask,
    }

train_dataset = preprocess_data(train_data)
val_dataset = preprocess_data(val_data)

Find Televisions options Zaghouan in Gabs Sud with a size of 55 and brand Apple.
Find Laptop options Monastir in Bab Bhar with a size of 42 and brand Azuz.


In [ ]:
train_dataset

In [ ]:


# Fine-tuning parameters
batch_size = 64
num_epochs = 1
learning_rate = 1e-5

# Prepare the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
# Set the device (CPU or GPU) for inference
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
# Fine-tuning loop
model.train()
for epoch in range(num_epochs):
    epoch_loss = 0.0

    for i in tqdm(range(0, len(train_dataset['input_ids']), batch_size)):
        batch_input_ids = train_dataset['input_ids'][i:i+batch_size].to(model.device)
        batch_attention_mask = train_dataset['attention_mask'][i:i+batch_size].to(model.device)
        batch_target_ids = train_dataset['target_ids'][i:i+batch_size].to(model.device)
        batch_target_attention_mask = train_dataset['target_attention_mask'][i:i+batch_size].to(model.device)

        outputs = model(
            input_ids=batch_input_ids,
            attention_mask=batch_attention_mask,
            labels=batch_target_ids,
            decoder_attention_mask=batch_target_attention_mask,
            return_dict=True
        )

        logits = outputs.logits
        loss = loss_fn(logits.view(-1, logits.shape[-1]), batch_target_ids.view(-1))
        epoch_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for i in range(0, len(val_dataset['input_ids']), batch_size):
            # Validation steps...
            # Compute validation loss
            val_loss += loss.item()

    print(f'Epoch {epoch+1} loss: {epoch_loss:.4f}, val_loss: {val_loss:.4f}')
    
# Save the fine-tuned model
model.save_pretrained('fine_tuned_model')
tokenizer.save_pretrained('fine_tuned_model')


100%|██████████| 5198/5198 [1:25:21<00:00,  1.01it/s]


Epoch 1 loss: 56.1641, val_loss: 0.0577


('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/spiece.model',
 'fine_tuned_model/added_tokens.json')

In [ ]:
import torch
import re
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the fine-tuned model and tokenizer
#tokenizer = T5Tokenizer.from_pretrained('/kaggle/working/fine_tuned_model')
#model = T5ForConditionalGeneration.from_pretrained('/kaggle/working/fine_tuned_model')

# Define the input text
input_text = "Find Televisions options Zaghouan in Gabs Sud with a size of 55 and brand Apple."

'''def preprocess_text(text):
    if not isinstance(text, (str, bytes)):
        text = str(text)
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    return text'''

# Preprocess the input text
#preprocessed_input = preprocess_text(input_text)

# Tokenize the input text
input_encoding = tokenizer.encode(input_text, padding=True, truncation=True, return_tensors='pt').to(device)

# Generate the output text
output_ids = model.generate(input_ids=input_encoding, max_length=50)
output_text = tokenizer.decode(output_ids.squeeze(), skip_special_tokens=True)

# Print the output text
print("Generated Text:", output_text)

Generated Text: What Televisions options are available Zaghouan in Gabs Sud with a size of 55 and brand Apple?


Data aug


In [ ]:
input_prompts=df['input']

In [ ]:
input_prompts[0]

'Find Televisions options Tunisia in Carthage with a size of 32 and brand Samsung.'

In [ ]:
generated_questions = []

In [ ]:
for input_prompt in tqdm(input_prompts):
    # Preprocess the input prompt
    #preprocessed_input = preprocess_text(input_prompt)
    if(input_prompt == input_prompts[0]):
      print(input_prompt)
    # Tokenize the input prompt
    input_encoding = tokenizer.encode(input_prompt, padding=True, truncation=True, return_tensors='pt').to(device)
    
    # Generate the output text
    output_ids = model.generate(input_ids=input_encoding, max_length=50)
    output_text = tokenizer.decode(output_ids.squeeze(), skip_special_tokens=True)
    
    # Add the generated question to the list
    generated_questions.append(output_text)


  0%|          | 0/415800 [00:00<?, ?it/s]

Find Televisions options Tunisia in Carthage with a size of 32 and brand Samsung.


  1%|          | 2236/415800 [17:41<54:31:09,  2.11it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:10                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1515 in generate        │
│                                                                                                  │
│   1512 │   │   │   │   )                                                                         │
│   1513 │   │   │                                                                                 │
│   1514 │   │   │   # 11. run greedy search                                                       │
│ ❱ 1515 │   │   │   return self.greedy_search(                                                    │
│   1516 │   │   │   │   input_ids,                                                                │
│   1517 │   │   │   │   logits_processor=logits_processor,                                        │
│   1518 │   │   │   │   stopping_criteria=stopping_criteria,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2332 in greedy_search   │
│                                                                                                  │
│   2329 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2330 │   │   │                                                                                 │
│   2331 │   │   │   # forward pass to get next token                                              │
│ ❱ 2332 │   │   │   outputs = self(                                                               │
│   2333 │   │   │   │   **model_inputs,                                                           │
│   2334 │   │   │   │   return_dict=True,                                                         │
│   2335 │   │   │   │   output_attentions=output_attentions,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used    

In [ ]:
import pandas as pd

# Create a new dataframe with the generated questions and input prompts
new_dataframe = pd.DataFrame({'input': input_prompts[:2248], 'output': generated_questions})


In [ ]:
new_dataframe

,input,output
0,Find Televisions options Tunisia in Carthage w...,What Televisions options are available Tunisia...
1,Find Televisions options Tunisia in Carthage w...,What Televisions options are available Tunisia...
2,Find Televisions options Tunisia in Carthage w...,What Televisions options are available Tunisia...
3,Find Televisions options Tunisia in Carthage w...,What Televisions options are available Tunisia...
4,Find Televisions options Tunisia in Carthage w...,What Televisions options are available Tunisia...
...,...,...
2243,Find Telephones options Tunisia in Béni Khiar ...,What Telephones options are available Tunisia ...
2244,Find Telephones options Tunisia in Béni Khiar ...,What Telephones options are available Tunisia ...
2245,Find Telephones options Tunisia in Béni Khiar ...,What Telephones options are available Tunisia ...
2246,Find Telephones options Tunisia in Béni Khiar ...,What Telephones options are available Tunisia ...


In [ ]:
# Concatenate the old and new dataframes
combined_dataframe = pd.concat([df, new_dataframe], axis=0)

In [ ]:
combined_dataframe.describe()

,input,output
count,418048,418048
unique,415800,415800
top,Find Televisions options Tunisia in Carthage w...,What Televisions options are available Tunisia...
freq,2,3


Fine-tuning

In [ ]:
combined_dataframe = combined_dataframe.sample(frac=0.2)

In [ ]:
combined_dataframe.describe()

,input,output
count,83610,83610
unique,83510,83502
top,Find Tablets options Tunisia in Bizerte Nord w...,What options are available Tunisia in Nabeul w...
freq,2,2


**Now we need to understand the NL Queries**